In [7]:
from collections import namedtuple
import random
import re
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# Подготавливаем данные 

In [2]:
# Предполагаем, что у каждого класса есть свой класс и в каждом классе проставляем номер курса класса
MAX_COUNT_DAYS = 6
MAX_COUNT_LESSONS = 8

CLASS_TABLE = np.zeros(MAX_COUNT_DAYS * MAX_COUNT_LESSONS)

CLASSES = {
           '1а':['ИЗО',            1],
           '1б':['ИЗО',            1],
           '2а':['ИЗО',            1],
           '2б':['ИЗО',            1],
           '3а':['ИЗО',            1],
           '3б':['ИЗО',            1],
           '4а':['ИЗО',            1],
           '4б':['ИЗО',            1],
           '5а':['Математика',     5, # 2
                 'Русский',        6, # 2
                 'Литература',     3, # 1.5
                 'ИЗО',            1, # 1
                 'Музыка',         1, # 1
                 'Физкультура',    2, # 1
                 'Труд',           2, # 1
                 'История',        2, # 1.5
                 'География',      2, # 1.5
                 'Информатика',    1, # 1.5
                 'Немецкий язык',  2, # 1.5
                 'Биология',       1  # 1.5 
                ], 
           '5б':['Математика',     5,
                 'Русский',        6,
                 'Литература',     3,
                 'ИЗО',            1,
                 'Музыка',         1,
                 'Физкультура',    2,
                 'Труд',           2,
                 'История',        2,
                 'География',      2,
                 'Информатика',    1,
                 'Немецкий язык',  2,
                 'Биология',       1
                ],
           '6а':['Математика',      5, # 2
                 'Русский',         7, # 2
                 'Литература',      3, # 1.5
                 'ИЗО',             1, # 1
                 'Музыка',          1, # 1
                 'Физкультура',     3, # 1
                 'Труд',            2, # 1
                 'История',         2, # 1.5
                 'География',       1, # 1.5
                 'Информатика',     1, # 1.5
                 'Немецкий язык',   3, # 1.5
                 'Французский язык',2, # 1.5
                 'Общестовазнание', 1, # 1.5
                 'Биология',        1  # 1.5
                ], 
           '6б':['Математика',       5,
                 'Русский',          7,
                 'Литература',       3,
                 'ИЗО',              1,
                 'Музыка',           1,
                 'Физкультура',      3,
                 'Труд',             2,
                 'История',          2,
                 'География',        1,
                 'Информатика',      1,
                 'Немецкий язык',    3,
                 'Французский язык', 2,
                 'Общестовазнание',  1,
                 'Биология',         1
                ],
           '7а':['Алгебра',          3, # 2
                 'Геометрия',        2, # 2
                 'Русский',          5, # 2
                 'Литература',       2, # 1.5
                 'ИЗО',              1, # 1
                 'Музыка',           1, # 1
                 'Физкультура',      3, # 1
                 'Труд',             2, # 1
                 'История',          3, # 2
                 'География',        2, # 2
                 'Информатика',      1, # 2
                 'Немецкий язык',    2, # 2
                 'Французский язык', 2, # 2
                 'Общестовазнание',  1, # 2
                 'Биология',         1, # 2
                 'Физика',           2, # 2
                 'ОБЖ',              1  # 1
                ],
           '7б':['Алгебра',          3,
                 'Геометрия',        2,
                 'Русский',          5,
                 'Литература',       2,
                 'ИЗО',              1,
                 'Музыка',           1,
                 'Физкультура',      3,
                 'Труд',             2,
                 'История',          3,
                 'География',        2,
                 'Информатика',      1,
                 'Немецкий язык',    2,
                 'Французский язык', 2,
                 'Общестовазнание',  1,
                 'Биология',         1,
                 'Физика',           2, # 2
                 'ОБЖ',              1
                ],
           '8а':['Алгебра',          4,
                 'Геометрия',        2,
                 'Русский',          5,
                 'Литература',       2,
                 'ИЗО',              1,
                 'Музыка',           1,
                 'Физкультура',      3,
                 'Труд',             1,
                 'История',          2,
                 'География',        2,
                 'Информатика',      1,
                 'Немецкий язык',    3,
                 'Общестовазнание',  1,
                 'Биология',         2,
                 'Физика',           2,
                 'ОБЖ',              1,
                 'Химия',            2 # 2
                ],
           '9а':['Алгебра',         4,
                 'Геометрия',       2,
                 'Русский',         5,
                 'Литература',      3,
                 #'ИЗО',            1
                 #'Музыка',       # 1
                 'Физкультура',     3,
                 #'Труд',         # 1
                 'История',         3,
                 'География',       2,
                 'Информатика',     1,
                 'Немецкий язык',   3,
                 'Французский язык', 1,
                 'Общестовазнание',  1,
                 'Биология',        2,
                 'Физика',          3,
                 'ОБЖ',             1,
                 'Химия',           2
                ]          
          }
CLASSROOMS = [i for i in CLASSES]
TEACHERS = ['Чуркина Наталья Аркадьевна',      # Немецкий язык __ французский
            'Просвирин Сергей Дмитриевич',     # История __ Технология
            'Митрохина Альбина Николаевна',    # Математика
            'Захарова Ирина Ивановна',         # Математика __ Физика
            'Суханова Анна Львовна',           # Биологии __ Химии
            'Ершова Светлана Николаевна',      # Физра __ География
            'Селезнева Светлана Валерьевна',   # История Информатика
            'Самарина Валентина Ивановна',     # Немецкий язык __ Французский
            'Сухарева Оксана Николаевна',      # Русский язык __ Литература
            'Меликаева Налия Кирилловна',      # Изо
            'Сидорова Софья Александровна',    # Русский язык __ Литература __ География
            'Соловьева Розалия Андреевна',     # Русский язык __ Литература
            'Тукова Елена Николаевна',         # Технология __ ОБЖ
            'Похвалий Екатерина Михайловна',   # Музыка
            'Сорокина Анастасия Александровна' # Русский язык __ Литература
           ]

lst = ['ItemName','TeacherFIO','Class','count_lessons']
courseDescription = namedtuple('courseDescription', lst)

COURSES_1a = {}
COURSES_1б = {}
COURSES_2a = {}
COURSES_2б = {}
COURSES_3a = {}
COURSES_3б = {}
COURSES_4a = {}
COURSES_4б = {}
COURSES_5a = {}
COURSES_5б = {}
COURSES_6a = {}
COURSES_6б = {}
COURSES_7а = {}
COURSES_7б = {}
COURSES_8а = {}
COURSES_9а = {}


for class_, items in CLASSES.items():
    KEY_COURSE = 1
    for count, item in enumerate( zip(items[0::2], items[1::2]) ):
        lesson = item[0]
        count_lessons_per_week = item[1]
                                 
        if class_ in ['5а','5б','6а','6б','7а'] and lesson in ['Французский язык','Немецкий язык']:
            teacher = TEACHERS[7]
        elif lesson in ['Французский язык','Немецкий язык']:
            teacher = TEACHERS[0]
        
        if class_ in ['5а','5б','6а','6б'] and lesson == 'История':
            teacher = TEACHERS[1]
        elif lesson == 'История':
            teacher = TEACHERS[6]
        
        if class_ in ['5а','5б','6а'] and lesson == 'Труд':
            teacher = TEACHERS[12]
        elif lesson == 'Труд':
            teacher = TEACHERS[1]                         
        
        if (class_ in ['7а','8а','9а'] and lesson in ['Алгебра','Геометрия']) or (lesson == 'Физика'):
            teacher = TEACHERS[3]
        elif lesson in ['Алгебра','Геометрия','Математика']:
            teacher = TEACHERS[2]
            
        if lesson in ['Биология','Химия']:
            teacher = TEACHERS[4]
                                 
        if lesson == 'ИЗО':
            teacher = TEACHERS[9]
                            
        if lesson == 'Музыка':
            teacher = TEACHERS[-2]
        
        if lesson == 'ОБЖ':
            teacher = TEACHERS[-3]
        
        if lesson == 'Информатика':
            teacher = TEACHERS[6]
                                 
        if (lesson == 'Физкультура') or (lesson == 'География' and class_ in ['5а','5б']):
            teacher = TEACHERS[5]
        elif (lesson == 'География') or (lesson in ['Русский', 'Литература'] and class_  == '5а'):
            teacher = TEACHERS[10]
                                 
        if lesson in ['Русский', 'Литература'] and class_  in ['5б','6а']:
            teacher = TEACHERS[8]
        
        if lesson in ['Русский', 'Литература'] and class_  in ['6б','9а']:
            teacher = TEACHERS[-4]
        
        if lesson in ['Русский', 'Литература'] and class_  in ['7а','7б','8а']:
            teacher = TEACHERS[-1]
                                 
        f = courseDescription(item, teacher, class_, count_lessons_per_week)  
        
        # Создаем словари для СЛОВАРЯ
        if class_ == '1а':
            COURSES_1a.update({KEY_COURSE :  f})
        if class_ == '2а':
            COURSES_2a.update({KEY_COURSE :  f})
        if class_ == '3а':
            COURSES_3a.update({KEY_COURSE :  f})
        if class_ == '4а':
            COURSES_4a.update({KEY_COURSE :  f})

        if class_ == '1б':
            COURSES_1б.update({KEY_COURSE :  f})
        if class_ == '2б':
            COURSES_2б.update({KEY_COURSE :  f})
        if class_ == '3б':
            COURSES_3б.update({KEY_COURSE :  f})
        if class_ == '4б':
            COURSES_4б.update({KEY_COURSE :  f})

        if class_ == '5а':
            COURSES_5a.update({KEY_COURSE :  f})   
        
        if class_ == '5б':
#            COURSES_5б.update({0: courseDescription('Ничего', 'Ничего', 'Ничего', 20) })
            COURSES_5б.update({KEY_COURSE :  f})
            
            
        if class_ == '6а':
#            COURSES_6a.update({0: courseDescription('Ничего', 'Ничего', 'Ничего', 15) })
            COURSES_6a.update({KEY_COURSE :  f})
            
            
        if class_ == '6б':
#            COURSES_6б.update({0: courseDescription('Ничего', 'Ничего', 'Ничего', 15) })
            COURSES_6б.update({KEY_COURSE :  f})
            
        
        if class_ == '7а':
#            COURSES_7а.update({0: courseDescription('Ничего', 'Ничего', 'Ничего', 14) })
            COURSES_7а.update({KEY_COURSE :  f})
            
        if class_ == '7б':
#            COURSES_7б.update({0: courseDescription('Ничего', 'Ничего', 'Ничего', 14) })
            COURSES_7б.update({KEY_COURSE :  f})
            
        if class_ == '8а':
#            COURSES_8а.update({0: courseDescription('Ничего', 'Ничего', 'Ничего', 13) })
            COURSES_8а.update({KEY_COURSE :  f})
            
        if class_ == '9а':
#            COURSES_9а.update({0: courseDescription('Ничего', 'Ничего', 'Ничего', 12) })
            COURSES_9а.update({KEY_COURSE :  f})
            

        KEY_COURSE += 1
#COURSES.update({0: courseDescription('Ничего', 'Ничего', 'Ничего', 0) })
# len(COURSES)
COURSES = [
           COURSES_1a,
           COURSES_1б,
           COURSES_2a,
           COURSES_2б,
           COURSES_3a,
           COURSES_3б,
           COURSES_4a,
           COURSES_4б,
           COURSES_5a,
           COURSES_5б,
           COURSES_6a,
           COURSES_6б,
           COURSES_7а,
           COURSES_7б,
           COURSES_8а,
           COURSES_9а]

In [42]:
COURSES

[{1: courseDescription(ItemName=('ИЗО', 1), TeacherFIO='Меликаева Налия Кирилловна', Class='1а', count_lessons=1)},
 {1: courseDescription(ItemName=('ИЗО', 1), TeacherFIO='Меликаева Налия Кирилловна', Class='1б', count_lessons=1)},
 {1: courseDescription(ItemName=('ИЗО', 1), TeacherFIO='Меликаева Налия Кирилловна', Class='2а', count_lessons=1)},
 {1: courseDescription(ItemName=('ИЗО', 1), TeacherFIO='Меликаева Налия Кирилловна', Class='2б', count_lessons=1)},
 {1: courseDescription(ItemName=('ИЗО', 1), TeacherFIO='Меликаева Налия Кирилловна', Class='3а', count_lessons=1)},
 {1: courseDescription(ItemName=('ИЗО', 1), TeacherFIO='Меликаева Налия Кирилловна', Class='3б', count_lessons=1)},
 {1: courseDescription(ItemName=('ИЗО', 1), TeacherFIO='Меликаева Налия Кирилловна', Class='4а', count_lessons=1)},
 {1: courseDescription(ItemName=('ИЗО', 1), TeacherFIO='Меликаева Налия Кирилловна', Class='4б', count_lessons=1)},
 {1: courseDescription(ItemName=('Математика', 5), TeacherFIO='Митрохина

In [ ]:
# [{
#   'clas' : '1a',
#   'courses' : {1 : ['Французский язык', 'FIO', 1],
#                2 : ['Математика', 'FIO', 2],
#  
#               },
#  },
#  {}
#]
# 
# 
# 
# {},{},{},{},{},{}]

# Создание эталлонного расписания 

Поэтапное описание алгоритма:
1) Определяем изначальные параметры
    1.1 Порог на кол-во уроков в одном дне. 8 Стандартный порог.
    1.2 Параметр "стоимость предметов" (BASIC_LVL_LESSONS) . Определяется статическим словарем, где сложный предмет - 2 очка, средний - 1.5, легкий - 1.
    1.3 Параметр "кол-во уроков у класса". 
        1.3.1 На первом этапе определяем сколько дней в недели может учится класс - 5 или 6. По стандарту определяем, что 9 класс учится 6 дней. 
        1.3.2 общее кол-во уроков класса в неделю делим на кол-во дней из п.1.3.1 и округляем вниз.
        1.3.3 Число из п.1.3.2 умножаем на число из п.1.3.1 и вычитаем из него общее кол-во уроков. Получаем кол-во уроков, которое необходимо добавить к п.1.3.2, но так, чтобы порог по кол-ву уроков в одном дне не превышал порог из п. 1.1 .
        1.3.4 Добавление в данном алгоритме происходило по жадному алгоритму. Если у нас оставалось 2 занятия, то мы добавляем к п. 1.3.2 1 занятие в понедельник и во вторник. То есть жадный - добавляем уроки с понедельника.
        
2) Алгоритм построения эталлнного расписания
    2.1 Процесс сборки является итерационным. Итерация выглядит так: собираем расписание для класса -> собираем день -> собираем неделю.
    2.2 На этапе сборки расписания для класса учитываем ограничения:
        2.2.1 - Чтобы более 2 уроков не было в одном дне у одного класса
        2.2.2 - Чтобы учителя не вели урок одновременно в разных классах
        2.2.3 - Кол-во уроков в неделю должно совпадать с кол-вом заданным изначально.
    2.3 Сборка алгоритма начинается с минимального класса - с 5.
        
    

In [21]:
def greedy_alg(standart_lessons : int, add_lessons : int, basic_count_days : int, reverse : bool = False) -> list:
  """
  Function - greedy algorithm. Add remaining lessons, starting from monday if reverse = False else from friday or saturday.
  
  params:
    standart_lessons : int - mean count of lessons without remainder.
    add_lessons : int - remainder of lessons.
    basic_count_days : int - study days in class.
    reverse : bool (default - False) - if false then add lessons starting from monday else from friday or saturday.

  return - list.
  """
  
  count_lessons = [ standart_lessons + 1 if i <= add_lessons else standart_lessons for i in range(1, basic_count_days + 1) ]
  if reverse:
    count_lessons.reverse()
  return count_lessons


def random_alg(standart_lessons : int, add_lessons : int, basic_count_days : int) -> list:
  """
  Function - random algorithm. Add remainder lessons use random.
  
  params:
    standart_lessons : int - mean count of lessons without remainder.
    add_lessons : int - remainder of lessons.
    basic_count_days : int - study days in class.

  return - list.
  """

  count_lessons = [standart_lessons for i in range(1, basic_count_days + 1)]
  while add_lessons:
    idx = random.randint(0, len(count_lessons) - 1)
    add = random.randint(0, 1)
    if add and count_lessons[idx] == standart_lessons:
      add_lessons = add_lessons - 1 if add == 1 else add_lessons 
      count_lessons[idx] = standart_lessons + add

  return count_lessons


def middle_alg(standart_lessons : int, add_lessons : int, basic_count_days : int)-> list:
  """
  Function - middle algorithm. Add remainder lessons starting from wensday.
  
  params:
    standart_lessons : int - mean count of lessons without remainder.
    add_lessons : int - remainder of lessons.
    basic_count_days : int - study days in class.

  return - list.
  """

  count_lessons = [standart_lessons + 1 if add_lessons == basic_count_days else standart_lessons for i in range(1, basic_count_days + 1)]
  # Add wensday.
  if add_lessons == 1:
    count_lessons[2] = standart_lessons + 1
  # Add other days.
  if 1 < add_lessons < basic_count_days:
    for idx in range(1, add_lessons + 1):
      count_lessons[idx] = standart_lessons + 1
  return count_lessons


def get_result_alg(name_algorithm : str, standart_lessons : int, add_lessons : int, basic_count_days : int) -> list:
  """Choose which algorithm run depend on type_algorithm.
     
     params:
        type_algorithm : str - name of algorithm

     return list if not errors
  """

  types_algorithms = {'greedy' : greedy_alg,
                      'greedy_reverse' : greedy_alg,
                      'random' : random_alg,
                      'middle' : middle_alg
                      }
  try:
    func = types_algorithms[name_algorithm]
    if 'reverse' in name_algorithm:
      return func(standart_lessons, add_lessons, basic_count_days, True)
    else:
      return func(standart_lessons, add_lessons, basic_count_days)
  except:
    print(f'ERROR! We use name algorithm {name_algorithm}. But need {list(types_algorithms.keys())}')
    return None


def get_classes_standart_settings(BASIC_LVL_LESSONS : dict, BASIC_COUNT_STUDY_DAYS : list, name_algorithm : str) -> list:
  """
       Func calc two params:
       1 - Count lessons on every day for each class. For begginig classes get 4
           lessons and complexity - if one subject - 0.2 else ....
           For others algorithm calc in 2 steps:

            1) Find mean count lessons in every day for class:
              a) Calc all count lessons for class in week.
              b) Devide step а. by count study'es days class.
              c) Rounding floor step b. End step 1.

            2) Calc count lessons remainder and summarize it's lessons with step
             one use algorithms greedy, greedy_reverse, random, middle:
              a) From step 1.a minus step 1.b multiplication by count study days.
              b) Choose algorithm that place remainder lessons.
                - greedy : add lessons from monday, tueday, wensday and continue...
                - greedy_reverse : analogue greedy but reverse.
                - random : use only random place.
                - middle : starting add lessons from wensday next add tuesday, thursday, friday, monday (if 5 study days).

       2 - Mean complexity on day for class.
         1)  Sum of complexity all lessons class devide by count study days class.

       Params:
        BASIC_LVL_LESSONS - dict, price of lessons.
        BASIC_COUNT_STUDY_DAYS - list, count study days.
        name_algorithm - str, name algorithm.
  """

  # For every class define count of lessons by every study day
  CLASSES_STANDART_SETTINGS = []
  for clas, basic_count_days in zip(COURSES, BASIC_COUNT_STUDY_DAYS):

      number_class = list(clas.values())[0].Class
      number_class = int(re.findall('^[ 0-9]', number_class)[0])

      #  Load on begginer classes. It's bad - change! 
      if number_class <= 4:
          count_lessons = [4,4,4,4,4,0, 0.2]
      else:
          # Step 1 - define mean count lesson and how much need add lessons
          all_lessons = sum([i.count_lessons for i in clas.values()])
          standart_lessons = math.floor(all_lessons / basic_count_days)
          add_lessons = all_lessons - standart_lessons * basic_count_days
          
          # Step 2 - use algorithm destribute adding lessons
          count_lessons = get_result_alg(name_algorithm, standart_lessons, add_lessons, basic_count_days)
          if basic_count_days < 6:
              count_lessons.append(0)
          
          # Step 3 - calc mean load on class.
          count_lvl_lessons = [ BASIC_LVL_LESSONS[i.ItemName[0]] * i.count_lessons for i in clas.values() ]
          avg_lvl = sum(count_lvl_lessons) / basic_count_days
          count_lessons.append(avg_lvl)

      CLASSES_STANDART_SETTINGS.append(count_lessons)
  return CLASSES_STANDART_SETTINGS


def get_params_default():

    """ 
    Собрем все возможные параметы.
    """
    
    name_algorithm = 'greedy'
    # defalut - изначальная стоимость уроков. Разделяем урки на 3 уровня: Сложный (2), Средний (1.5), Легкий (1)
    BASIC_LVL_LESSONS = {
         'Алгебра' : 2,
         'Биология' : 1.5,
         'География' : 1.5,
         'Геометрия' : 2,
         'ИЗО' : 1,
         'Информатика' : 1.5,
         'История' : 1.5,
         'Литература' : 1.5,
         'Математика' : 2,
         'Музыка' : 1,
         'Немецкий язык' : 1.5,
         'ОБЖ' : 1,
         'Общестовазнание' : 1.5,
         'Русский' : 2,
         'Труд' : 1,
         'Физика' : 2,
         'Физкультура' : 1,
         'Французский язык' : 1.5,
         'Химия' : 2         
    }
    
    # default - кол-во учебных дней в классе. Начиная с 9 класса - 6 дней, остальные 5 дней.
    BASIC_COUNT_STUDY_DAYS = [6 if int(re.findall('^[ 0-9]', list(i.values())[0].Class)[0]) in [9,10,11] else 5 for i in COURSES]
    
    # Собираем информацию для каждого класса по каждому дню максимальное кол-во уроков в дне. Последним элементом сохраняем информацию о средней нагрузке в день.
    CLASSES_STANDART_SETTINGS = get_classes_standart_settings(BASIC_LVL_LESSONS, BASIC_COUNT_STUDY_DAYS, name_algorithm)
    
    # Собираем информацию о кол-ве уроков на каждый день. Нужно для проврки алгоритма
    COUNT_LESSONS_WEEK = []
    for day in range(6):
        COUNT_LESSONS_WEEK.append(sum([i[day] for i in CLASSES_STANDART_SETTINGS]))

    MIN_LVL_AVG_COST = 2
    MAX_LVL_AVG_COST = 2
    MAX_STUDY_LESSON = 8

    return BASIC_LVL_LESSONS, CLASSES_STANDART_SETTINGS, COUNT_LESSONS_WEEK, MIN_LVL_AVG_COST, MAX_LVL_AVG_COST, MAX_STUDY_LESSON
    

# CORE

In [40]:

class SchoolTable():
    """
    Use analitic logic, we try create algorithm, that made good school timetable for old school.
    """
    def __init__(self, 
                BASIC_LVL_LESSONS, 
                CLASSES_STANDART_SETTINGS, 
                COUNT_LESSONS_WEEK, 
                MIN_LVL_AVG_COST, 
                MAX_LVL_AVG_COST, 
                COURSES, 
                MAX_STUDY_LESSON):
        """
        Need to get info about current user, that post params for algorithm.

        params:
            BASIC_LVL_LESSONS : dict - cost of lessons.
            CLASSES_STANDART_SETTINGS : list - count lessons in day for class and mean complexity in day.
            COUNT_LESSONS_WEEK : list - count all lessons in every day for sshool.
        """

        self.BASIC_LVL_LESSONS = BASIC_LVL_LESSONS
        self.CLASSES_STANDART_SETTINGS = CLASSES_STANDART_SETTINGS
        self.COUNT_LESSONS_WEEK = COUNT_LESSONS_WEEK
        self.MIN_LVL_AVG_COST = MIN_LVL_AVG_COST
        self.MAX_LVL_AVG_COST = MAX_LVL_AVG_COST
        self.COURSES = COURSES
        self.MAX_STUDY_LESSON = MAX_STUDY_LESSON

        self.RESULT_TIMETABLE = []

    def get_timetable(self, count_of_increment = 1000) -> list:
        """ 
        It's main function, that start algorithm.    
        """
        
        increment = 0
        while True:
            if increment > count_of_increment:
                break

            for idx_day, count_lessons_day in enumerate(self.COUNT_LESSONS_WEEK):
                increment += 1  
                timetable_day = self.get_timetable_day(idx_day, count_lessons_day)
                if not timetable_day or increment > count_of_increment:
                    break 

                self.RESULT_TIMETABLE[idx_day] = timetable_day
            
            if timetable_day:
                return self.RESULT_TIMETABLE
            else:
                self.RESULT_TIMETABLE = []
      
    def get_timetable_day(self, idx_day : int, count_lessons_day : int) -> list:

        """
        Function return 
        """

        count_lessons = 0
        for idx_clas in range(len(self.CLASSES_STANDART_SETTINGS)):
            timetable_class = self.insert_courses(idx_clas, idx_day)           
            if not timetable_class:
                return False
            
            #  Insert into timetable a new class
            if len(self.RESULT_TIMETABLE) == 0 or idx_clas == 0:
                self.RESULT_TIMETABLE.append([timetable_class])
            else:
                self.RESULT_TIMETABLE[idx_day].append(timetable_class)

            count_lessons += sum([0 if i == 0 else 1 for i in timetable_class])
                
        #assert count_lessons == count_lessons_day, 'кол-во уроков заданных в базе не совпадает с количеством добавленных уроков!'       
        #print(f'Get timetable class! \n {timetable_class}')
        return self.RESULT_TIMETABLE[idx_day]
        
    def insert_courses(self, idx_clas : int, idx_day : int, count_of_increment = 100) -> list:
        """
        Заходим в класс. Проставляем расписание класса за день.
        Рандомно ставим первый урок. После идет проверка на ограничение : Если урок уже был 2 раза проставлен, больше не    может учавствовать в этом дне.
        
        Нагрузка должна быть больше наименьшего порога. Если порог не достигнут или наоборот - превыше, то заново пересчитываем.
        Ограничения, которые поступают сюда, должны учитывать прошлые расчеты. То есть для каждого класса фиксировать использованные уроки.
        """
        
        count_lesson = self.CLASSES_STANDART_SETTINGS[idx_clas][idx_day]
        flag_constraint = False
        increment = 0
        courses = self.COURSES[idx_clas]
        
        while True:
            increment += 1
            # for current course
            class_course = []
            # access courses from all timetable
            access_courses = []
            # if in this day class hasn't lessons create list with zeros.
            if count_lesson == 0:
                return [0] * self.MAX_STUDY_LESSON
            # if increment out of count_of_increment then out.
            if increment >= count_of_increment:
                return False
                
            for lesson in range(0, self.MAX_STUDY_LESSON):
                #print(f' day {idx_day} clas {idx_clas} lesson {lesson}')
                if lesson <= count_lesson and count_lesson != 0:
                    if idx_day == 0 and idx_clas == 0 and lesson == 0:
                        class_course.append(random.choice(list(courses.keys())))
                    else:
                        access_courses = self.get_access_courses(lesson, idx_day, idx_clas, courses, class_course)
                        if len(access_courses) == 0 or len(class_course) == count_lesson: 
                            class_course.append(0)
                        else:
                            class_course.append(random.choice(access_courses))   
                else:
                    class_course.append(0)
            
            flag_constraint = self.constrains(idx_clas, class_course)
            if flag_constraint:
                return class_course

    def get_access_courses(self, idx_lesson : int, idx_day : int, idx_clas : int, courses : dict, class_course : list) -> list :
        """ 
        Moduls of check access courses.
        """       
        # out of lessons
        access_courses = self.check_count_lesson_in_day(courses, class_course)
        # out of courses
        access_courses = self.out_cours(idx_day, idx_clas, class_course, access_courses, courses)   
        # check teachers
        access_courses = self.check_teachers(idx_day, idx_clas, idx_lesson, access_courses)

        return access_courses

    def check_count_lesson_in_day(self, courses : dict, class_course : list, parametr : int = 2) -> list:
        """
        Out of lesson in day if more then parametr
        """
        access_courses = list(courses.keys())
        
        # уберем курсы, которые 2 раза были в этом дне
        for course in set(class_course):
            if class_course.count(course) >= parametr and course != 0:
                access_courses.remove(course)

        return access_courses

    def out_cours(self, idx_day : int, idx_clas : int, class_course : list, access_courses : list, courses : dict):
        """
        Out corses, that haven't access on count in week yet.
        """

        if idx_day == 0:
            used_course = class_course
        else:
            used_course = [course for id_day, day in enumerate(self.RESULT_TIMETABLE) if id_day < idx_day for course in day[idx_clas]] + class_course
        
        for course in set(used_course):
            if course == 0 or course not in list(courses.keys()):
                continue
            if used_course.count(course) >= courses[course].count_lessons:
                try:
                    access_courses.remove(course)
                except:
                    continue
        return access_courses

    def check_teachers(self, idx_day : int, idx_clas : int, idx_lesson : int, access_courses : list) -> list:
        """
        Check teachers.
        """

        if idx_clas == 0:
            pass
        else:                
            courses_in_lesson = [value[idx_lesson] for idx, value in enumerate(self.RESULT_TIMETABLE[idx_day]) if idx != idx_clas]
            interseption_teachers = set([i.TeacherFIO for i in self.COURSES[idx_clas].values()]) & set([self.COURSES[idx][course].TeacherFIO for idx, course in enumerate(courses_in_lesson) if course != 0])

            if len(interseption_teachers) > 0:
                for course_key, course_value in self.COURSES[idx_clas].items():
                    if course_value.TeacherFIO in interseption_teachers:
                        try:
                            access_courses.remove(course_key)
                        except:
                            continue
        
        return access_courses

    def constrains(self, idx_clas :int, class_course : list) -> bool:
        """
        Check cost of study for class.
        """
        
        copy_class_course = class_course.copy()
        while True:
            try:
                copy_class_course.remove(0)
            except:
                break
                
        cost = sum([self.BASIC_LVL_LESSONS[ COURSES[idx_clas][course].ItemName[0] ] for course in copy_class_course])       
        avg_cost = round(self.CLASSES_STANDART_SETTINGS[idx_clas][-1])

        if avg_cost - self.MIN_LVL_AVG_COST <= cost <= avg_cost + self.MAX_LVL_AVG_COST:
            return True
        else:
            return False

In [41]:
BASIC_LVL_LESSONS, CLASSES_STANDART_SETTINGS, COUNT_LESSONS_WEEK, MIN_LVL_AVG_COST, MAX_LVL_AVG_COST, MAX_STUDY_LESSON = get_params_default()
timetable = SchoolTable(BASIC_LVL_LESSONS, CLASSES_STANDART_SETTINGS, COUNT_LESSONS_WEEK, MIN_LVL_AVG_COST, MAX_LVL_AVG_COST, COURSES, MAX_STUDY_LESSON )
school_table = timetable.get_timetable()
school_table

[[[1, 0, 0, 0, 0, 0, 0, 0],
  [0, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 1, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [5, 9, 8, 11, 2, 8, 0, 0],
  [8, 12, 7, 1, 2, 1, 0, 0],
  [13, 9, 14, 6, 10, 11, 5, 0],
  [14, 3, 13, 7, 5, 2, 9, 0],
  [3, 9, 3, 4, 2, 6, 13, 0],
  [14, 17, 11, 6, 4, 4, 15, 0],
  [9, 2, 7, 14, 14, 9, 5, 0],
  [5, 9, 3, 8, 5, 7, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 0, 0, 0],
  [0, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 1, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 0],
  [11, 1, 3, 6, 7, 12, 0, 0],
  [6, 10, 1, 2, 3, 4, 0, 0],
  [8, 8, 11, 12, 6, 12, 11, 0],
  [2, 2, 3, 7, 4, 3, 10, 0],
  [9, 16, 17, 16, 9, 2, 15, 0],
  [10, 7, 7, 1, 8, 3, 10, 0],
  [16, 17, 6, 10, 10, 11, 12, 0],
  [1, 7, 2, 3, 13, 9, 0, 0]],
 [[0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0

In [10]:
def get_grah_timetable(timetable : list) -> None:
    
    import pandas as pd
    
    result_df = pd.DataFrame()
    for day in timetable:        
        df_day = pd.DataFrame()
        for idx, clas in enumerate(day):
            temp_df = pd.DataFrame([COURSES[idx][values].ItemName[0] if values != 0 else 0 for values in clas], columns = [f'{idx}'])
            df_day = pd.concat([df_day, temp_df], axis = 1)

        result_df = pd.concat([result_df, df_day])  

    return result_df

def get_grah_teachertimetable(timetable : list) -> None:
    
    import pandas as pd
    
    result_df = pd.DataFrame()
    for day in timetable:        
        df_day = pd.DataFrame()
        for idx, clas in enumerate(day):
            temp_df = pd.DataFrame([COURSES[idx][values].TeacherFIO if values != 0 else 0 for values in clas], columns = [f'{idx}'])
            df_day = pd.concat([df_day, temp_df], axis = 1)

        result_df = pd.concat([result_df, df_day])  

    return result_df

df = get_grah_timetable(timetable)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,ИЗО,0,0,0,0,0,0,0,Музыка,Математика,Физкультура,Литература,Литература,ОБЖ,Общестовазнание,Алгебра
1,0,ИЗО,0,0,0,0,0,0,Физкультура,Немецкий язык,Русский,История,Литература,Немецкий язык,Физика,Русский
2,0,0,ИЗО,0,0,0,0,0,Биология,Математика,Литература,Общестовазнание,ОБЖ,История,Труд,Физика
3,0,0,0,ИЗО,0,0,0,0,География,Биология,Информатика,Труд,Алгебра,Музыка,Литература,Французский язык
4,0,0,0,0,0,0,0,0,Математика,Информатика,Французский язык,Музыка,География,Немецкий язык,Алгебра,Русский
5,0,0,0,0,0,0,0,0,История,Литература,Математика,Литература,Биология,Литература,ОБЖ,Общестовазнание
6,0,0,0,0,0,0,0,0,0,0,Физкультура,Информатика,Французский язык,Общестовазнание,География,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,ИЗО,0,0,0,География,Русский,Труд,Литература,Информатика,Русский,Музыка,География
1,0,0,0,0,0,ИЗО,0,0,Математика,История,Немецкий язык,Биология,Музыка,Русский,Немецкий язык,Физкультура


In [7]:
get_grah_teachertimetable(timetable)

,0,1,2,3,4,5,6,7
0,Митрохина Альбина Николаевна,Тукова Елена Николаевна,Самарина Валентина Ивановна,Похвалий Екатерина Михайловна,Сорокина Анастасия Александровна,Чуркина Наталья Аркадьевна,Суханова Анна Львовна,Соловьева Розалия Андреевна
1,Меликаева Налия Кирилловна,Суханова Анна Львовна,Самарина Валентина Ивановна,Митрохина Альбина Николаевна,Сидорова Софья Александровна,Захарова Ирина Ивановна,Селезнева Светлана Валерьевна,Соловьева Розалия Андреевна
2,Митрохина Альбина Николаевна,Ершова Светлана Николаевна,Меликаева Налия Кирилловна,Просвирин Сергей Дмитриевич,Сидорова Софья Александровна,Сорокина Анастасия Александровна,Чуркина Наталья Аркадьевна,Тукова Елена Николаевна
3,Просвирин Сергей Дмитриевич,Похвалий Екатерина Михайловна,Митрохина Альбина Николаевна,Соловьева Розалия Андреевна,Селезнева Светлана Валерьевна,Тукова Елена Николаевна,Ершова Светлана Николаевна,Захарова Ирина Ивановна
4,Ершова Светлана Николаевна,Самарина Валентина Ивановна,Сухарева Оксана Николаевна,Сидорова Софья Александровна,Просвирин Сергей Дмитриевич,Чуркина Наталья Аркадьевна,Захарова Ирина Ивановна,Селезнева Светлана Валерьевна
5,Тукова Елена Николаевна,Митрохина Альбина Николаевна,Ершова Светлана Николаевна,Самарина Валентина Ивановна,Просвирин Сергей Дмитриевич,Сорокина Анастасия Александровна,Суханова Анна Львовна,Чуркина Наталья Аркадьевна
6,0,0,Суханова Анна Львовна,Митрохина Альбина Николаевна,Селезнева Светлана Валерьевна,Ершова Светлана Николаевна,Сорокина Анастасия Александровна,Чуркина Наталья Аркадьевна
7,0,0,0,0,0,0,0,Ершова Светлана Николаевна
0,Селезнева Светлана Валерьевна,Митрохина Альбина Николаевна,Самарина Валентина Ивановна,Ершова Светлана Николаевна,Сорокина Анастасия Александровна,Чуркина Наталья Аркадьевна,Суханова Анна Львовна,Захарова Ирина Ивановна
1,Похвалий Екатерина Михайловна,Митрохина Альбина Николаевна,Тукова Елена Николаевна,Просвирин Сергей Дмитриевич,Самарина Валентина Ивановна,Сорокина Анастасия Александровна,Чуркина Наталья Аркадьевна,Ершова Светлана Николаевна


# Модули генетического алгоритма 

In [71]:
def get_timetable_structure_for_ga(timetable : list) -> list:
    import pandas as pd
    
    result_df = pd.DataFrame()
    for day in timetable:        
        df_day = pd.DataFrame()
        for idx, clas in enumerate(day):
            temp_df = pd.DataFrame(clas, columns = [f'{idx}'])
            df_day = pd.concat([df_day, temp_df], axis = 1)

        result_df = pd.concat([result_df, df_day])  

    return [j for i in result_df.values.tolist() for j in i]

timetable = get_timetable()
get_timetable_structure_for_ga(timetable)

2it [00:00, 15.48it/s]

Посчитаем кол-во уроков в будни для класса : [4, 4, 4, 4, 4, 0, 0.2] 

Посчитаем кол-во уроков в будни для класса : [4, 4, 4, 4, 4, 0, 0.2] 

Посчитаем кол-во уроков в будни для класса : [4, 4, 4, 4, 4, 0, 0.2] 

Посчитаем кол-во уроков в будни для класса : [4, 4, 4, 4, 4, 0, 0.2] 

Посчитаем кол-во уроков в будни для класса : [4, 4, 4, 4, 4, 0, 0.2] 

Посчитаем кол-во уроков в будни для класса : [4, 4, 4, 4, 4, 0, 0.2] 

Посчитаем кол-во уроков в будни для класса : [4, 4, 4, 4, 4, 0, 0.2] 

Посчитаем кол-во уроков в будни для класса : [4, 4, 4, 4, 4, 0, 0.2] 

Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 0, 8.9] 

Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 0, 8.9] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 0, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 0, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 0, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 0, 10.6] 

По

3it [00:00, 10.42it/s]

REMOVE COURSE!
ACCESS COURSES []
DAY 2 CLASS 7 LESSON 2
CLASS COURSE [0, 0]
REMOVE COURSE!
ACCESS COURSES []
DAY 2 CLASS 7 LESSON 3
CLASS COURSE [0, 0, 0]
REMOVE COURSE!
ACCESS COURSES []
DAY 2 CLASS 7 LESSON 4
CLASS COURSE [0, 0, 0, 0]
ACCESS COURSES [1]
DAY 2 CLASS 7 LESSON 5
DAY 2 CLASS 7 LESSON 6
DAY 2 CLASS 7 LESSON 7
AVG COST 0
COST 0
CLASS!  [0, 0, 0, 0, 0, 0, 0, 0]
DAY 2 CLASS 8 LESSON 0
CLASS COURSE []
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
ACCESS COURSES [1, 2, 3, 4, 6, 7, 8, 11]
DAY 2 CLASS 8 LESSON 1
CLASS COURSE [11]
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
ACCESS COURSES [1, 2, 3, 5, 7, 9]
DAY 2 CLASS 8 LESSON 2
CLASS COURSE [11, 5]
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
ACCESS COURSES [1, 2, 6, 7, 9, 11]
DAY 2 CLASS 8 LESSON 3
CLASS COURSE [11, 5, 9]
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE

4it [00:00,  8.81it/s]


DAY 3 CLASS 14 LESSON 4
CLASS COURSE [15, 3, 10, 12]
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
ACCESS COURSES [12, 14, 17]
DAY 3 CLASS 14 LESSON 5
CLASS COURSE [15, 3, 10, 12, 17]
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
ACCESS COURSES [1, 12, 13, 16]
DAY 3 CLASS 14 LESSON 6
CLASS COURSE [15, 3, 10, 12, 17, 12]
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
ACCESS COURSES [1, 3, 6, 7, 10, 14]
DAY 3 CLASS 14 LESSON 7
CLASS COURSE [15, 3, 10, 12, 17, 12, 6]
REMOVE COURSE!
ACCESS COURSES [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]
AVG COST 11
COST 11.5
CLASS!  [15, 3, 10, 12, 17, 12, 6, 0]
DAY 3 CLASS 15 LESSON 0
CLASS COURSE []
REMOVE COURSE!
REMOVE

5it [00:00,  5.07it/s]

REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
ACCESS COURSES [7, 16]
DAY 4 CLASS 13 LESSON 6
CLASS COURSE [13, 3, 4, 9, 10, 7]
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
ACCESS COURSES [1, 3, 8, 10, 16]
DAY 4 CLASS 13 LESSON 7
AVG COST 11
COST 9.0
DAY 4 CLASS 13 LESSON 0
CLASS COURSE []
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
ACCESS COURSES [7, 13, 17]
DAY 4 CLASS 13 LESSON 1
CLASS COURSE [7]
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
REMOVE COURSE!
ACCESS COURSES [3, 9, 13, 16, 17]
DAY 4 CLASS 13 LESSON 2
CLASS COURSE [7, 16]
REMOVE CO

6it [00:00,  7.05it/s]


[1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 12,
 7,
 12,
 9,
 2,
 14,
 9,
 5,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 6,
 8,
 7,
 12,
 2,
 3,
 11,
 4,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 7,
 1,
 3,
 3,
 6,
 8,
 12,
 2,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 8,
 7,
 3,
 14,
 3,
 14,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 7,
 13,
 5,
 11,
 16,
 3,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 10,
 5,
 11,
 2,
 17,
 7,
 10,
 9,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 13,
 15,
 16,
 7,
 14,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 6,
 10,
 2,
 15,
 3,
 10,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 12,
 6,
 8,
 15,
 10,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 8,
 4,
 3,
 10,
 2,
 12,
 17,
 14,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 10,
 11,
 8,
 6,
 3,
 2,
 16,
 12,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 12,
 6,
 7,
 11,
 3,
 9,
 8,
 9,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 11,
 1,
 8,
 4,
 9,
 7,
 12,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 12,
 1,
 7,
 12,
 4,
 

In [72]:
def create_individum() -> list:
    """
    Создание индивидума
    """
    individum = []
    for _ in CLASS_TABLE:
        for i in range(len(CLASSROOMS)):
            individum.append(np.random.randint(0, max(COURSES[i].keys())))
    return individum  

def create_individum() -> list:
    """
    Создание индивидума под новую логику.
    """
    timetable = get_timetable()
    individum = get_timetable_structure_for_ga(timetable)
    return individum 

### Создание популяции 

In [73]:
def create_population(SIZE_POPULATION : int) -> list:
    """
    # Создадим популяцию индивидумов
    """
    return [create_individum() for _ in range(SIZE_POPULATION)]

### Функция, которую будем минимизировать 

In [75]:
def cost(individum : list) -> int:
    """
    Определим фнкцию стоимости индивидума, которую в будущем будем минимизировать
    """
    value = hardConstraintsCountLessons(individum) + \
            hardConstraintsTeachers(individum) + \
            liteConstrainsLessonsPlace(individum) + \
            liteConstraintsTeachersPlace(individum)
    return value

#### Модули функции минимизации 

In [76]:
def hardConstraintsCountLessons(individum : list) -> int:
    """
    # 1. hardConstraintsCountLessons - функция дает штраф, если мы поставили больше или меньше занятий по курсу
    """
    count_violation = []
    step = len(COURSES)
    
    for key, curs in enumerate(COURSES):
        info_course = []
        start_step = 0
    # Курсы которые содержатся в индивидуме
        while start_step < len(individum):
            info_course.append(individum[start_step: start_step + step][key])
            start_step += step
            
        count_lesson = [val.count_lessons for val in curs.values()] 
        for idx_course in range(1, max(curs.keys()) + 1):
            count_violation.append(abs(count_lesson[idx_course-1] - info_course.count(idx_course)))   
            
    print(f'dif count {sum(count_violation)}')
    return sum(count_violation) * 2


def hardConstraintsTeachers(individum : list) -> int:
    """
    # 2. hardConstraintsTeachers - если преподаватель стоит в один урок на двух разных курсах
    """
    count_violation = 0
    start_step = 0
    step = len(COURSES)
    
    # Курсы которые содержатся в индивидуме
    while start_step < len(individum):        
        tmp_individum = individum[start_step: start_step + step]
        start_step += step
        
        for key, curs in enumerate(COURSES):
            if tmp_individum[key] != 0:
                tmp_individum[key] = curs[tmp_individum[key]].TeacherFIO

        count_violation += sum([tmp_individum.count(teacher) for teacher in set(tmp_individum)]) - step
    
    print(f'dif count teacher {count_violation}')   
    return count_violation * 2


def liteConstrainsLessonsPlace(individum : list) -> int:
    """
    # 3. liteConstraintsLessonsPlace - если уроки расположены неоптимально.
    Неоптимально (дефолтные настройки) - 
     1. Первый урок физкульутра. 
     2. Когда всего занятий в неделю по предмету - 2шт., и они в одном дне. 
    """
    count_violation = 0
    start_step = 0
    step = len(COURSES)
    lesson = 0
    
    # Вариант 1 добавлен
    while start_step < len(individum):
        if lesson == 0:
            tmp_individum = individum[start_step: start_step + step]
            count_violation += sum([1 if tmp_individum[idx] != 0 and course[tmp_individum[idx]].ItemName[0] == 'Физкультура' else 0 for idx, course in enumerate(COURSES)])
            
        start_step += step
        lesson += 1
        
        if lesson == 8:
            lesson -= 8
    
    print(f'liteConstrainsLessonsPlace : {count_violation}')
    return count_violation


def liteConstraintsTeachersPlace(individum : list) -> int:
    """
    # 4. liteConstraintsTeachersPlace - если у учителей в расписании присутствуют пропуски.
    Эталон для учителя - 1 учитель + 1 день - нет пропусков в расписании.
    """
    
    count_violation = 0
    step = len(COURSES)
    
    for teacher in TEACHERS:
        result_df = [[None for _ in range(8)] for _ in range(5)]

        idx_day = 0
        idx_lesson = 0
        start_step = 0
        
        while start_step < len(individum):
            #print(f'DAY {idx_day} LESSON {idx_lesson}')
            if idx_day == 5:
                break
            
            tmp_individum = individum[ start_step : start_step + step ]
            for idx_clas, lesson in enumerate(tmp_individum):
                if lesson != 0 and COURSES[idx_clas][lesson].TeacherFIO == teacher:
                    result_df[idx_day][idx_lesson] = 0
                                            
            start_step += step
            if idx_lesson == 7:
                idx_day += 1
                idx_lesson = 0
                
                if result_df[idx_day - 1].count(0) in [1,0]:
                    count_violation += 0
                else:
                    g = result_df[idx_day - 1].copy()
                    start_index = g.index(0)
                    g.remove(0)
                    end_index = g.index(0) + 1

                    for i in range(start_index + 1, end_index):
                        count_violation += 1 if result_df[idx_day - 1][i] != 0 else 0
                
            idx_lesson += 1
        #print(f'TEACHER - {teacher} count violation : {count_violation}')
    print(f'liteConstraintsTeachersPlace : {count_violation}')                       
    return count_violation


def liteConstraintsTeachersWantToPlace(individum : list) -> int:
    """
    # 5. liteConstraintsTeachersWantToPlace - задается для каждого учителя на каждый день его пожелание по расписанию
    """
    return 0

### Функция отбора индивидума турнирным методом 

In [77]:
def select(population : list, COSTS : list, size_tournament : int) -> list:
    """
    # Функция, которая возвращает наилучшего индивидума - турнирный метод
    """
    
    k = len(population) / size_tournament
    best = []
    for _ in range(int(k)):        
        keys_lst = [i for i in range(len(COSTS))]
        idx_list = [random.choice(keys_lst) for _ in range(size_tournament)]       
        best.append(population[sorted([ [idx, COSTS[idx]] for idx in idx_list ], key = lambda x: x[1])[0][0]])
        
    best_costs = sorted([[ind, value] for ind, value in zip(best, [cost(ind) for ind in best])], key = lambda x: x[1])
    return [ind[0] for ind in best_costs] 

### Функция скрещивания индивидумов

In [78]:
def mate(population : list, best_population : list, P_CROSSOVER : float, SIZE_POPULATION :int, P_CROSSOVER_FIRST_PARENT : float) -> list:
    """
    # Функция, которая скрещивает популяцию
    """
    
    new_population = []
    while len(new_population) < SIZE_POPULATION:
        random.shuffle(population)
        for parent1, parent2 in zip(best_population, population):
            if random.random() < P_CROSSOVER:
# Равномерное скрещивание
#                 new_population.append([np.random.choice([par1, par2], 
#                                                         p = [P_CROSSOVER_FIRST_PARENT, 1 - P_CROSSOVER_FIRST_PARENT])\
#                                        for par1, par2 in zip(parent1, parent2)])
                new_population.append([random.choice([par1, par2]) for par1, par2 in zip(parent1, parent2)])
                new_population.append([par2 if add_par == par1 else par1 for add_par, par1, par2 in zip(new_population[-1], parent1, parent2)])
# Одноточечное скрещивание
#                 point = random.randint(0, len(parent1))
#                 new_population.append(parent1[:point] + parent2[point:])
#                 new_population.append(parent2[:point] + parent1[point:])
# Двуточечное скрещивание
#                 point_one = random.randint(0, len(parent1))
#                 point_two = random.randint(point_one, len(parent1))
#                 new_population.append(parent1[point_one:point_two] + parent2[:point_one] + parent2[point_two:])
#                 new_population.append(parent2[point_one:point_two] + parent1[:point_one] + parent1[point_two:])
            else:
                new_population.append(parent1)
                new_population.append(parent2)
        
    return new_population

### Мутация 

In [79]:
def mutate(population : list, P_MUTATE : float) -> list:
    """
    # Функция, которая проводит мутацию с индивидумами - Мутация обменом
    """
    new_population = []
    for mutant in population:
        if random.random() < P_MUTATE:
            for _ in range(2):
                mutant[random.randint(0, len(mutant) - 1)] = 0
        new_population.append(mutant)
    return new_population   

# Генетический алгоритм 

In [80]:
def main() -> list:
    """
    Модули генетического алгоритма, соединенные воедино.
    """
    
    # ****************************************************
    # Константы
    
    np.random.seed(56)  # Чтобы рандом не начинался с нуля 
    random.seed(56)     # Чтобы рандом не начинался с нуля 
    
    SIZE_POPULATION = 25   # Размер популяции
    MAX_GENERATIONS = 400   # Максимальное количество итераций
    P_CROSSOVER_FIRST_PARENT = 0.7 # Вероятность наследования от первого родителя
    
    P_CROSSOVER = 1  # Вероятность скрещивания
    P_MUTATION = 0.5 # Вероятность мутации
    
    # Константы конец
    # ****************************************************
    
    POPULATION = create_population(SIZE_POPULATION)
    
    min_costs = []
    avg_costs = []
    
    for i in tqdm(range(MAX_GENERATIONS)):
        
        COSTS = [cost(ind) for ind in POPULATION]
        NEW_POPULATION = select(POPULATION, COSTS, 4)
        POPULATION = mate(POPULATION, NEW_POPULATION, P_CROSSOVER, SIZE_POPULATION, P_CROSSOVER_FIRST_PARENT)
        POPULATION = mutate(POPULATION[:SIZE_POPULATION], P_MUTATION)
        
        min_costs.append(min(COSTS))
        avg_costs.append(np.mean(COSTS))
        
    
    plt.plot(min_costs, color='red')
    plt.plot(avg_costs, color='green')
    plt.xlabel('Поколение')
    plt.ylabel('Макс/средняя приспособленность')
    plt.title('Зависимость максимальной и средней приспособленности от поколения') 
    plt.show() 
    
    return POPULATION   

# Модуль без ген алгоритма

In [297]:
d = main()

5it [00:00, 327.22it/s]
5it [00:00, 286.72it/s]
5it [00:00, 433.43it/s]
5it [00:00, 398.69it/s]
5it [00:00, 378.04it/s]

Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 7, 11.3] 

Посчитаем кол-во уроков в будни для класса : [8, 7, 7, 7, 7, 12.0] 

Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для кл


5it [00:00, 444.45it/s]
5it [00:00, 293.88it/s]
5it [00:00, 275.77it/s]
5it [00:00, 267.68it/s]
5it [00:00, 339.05it/s]
5it [00:00, 369.14it/s]



Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 7, 11.3] 

Посчитаем кол-во уроков в будни для класса : [8, 7, 7, 7, 7, 12.0] 

Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 7, 11.3] 

Посчитаем кол-во уроков в будни для 

5it [00:00, 352.73it/s]
5it [00:00, 426.51it/s]
5it [00:00, 318.15it/s]
5it [00:00, 384.86it/s]
5it [00:00, 370.40it/s]
5it [00:00, 378.67it/s]
0it [00:00, ?it/s]

...

5it [00:00, 408.80it/s]
5it [00:00, 347.20it/s]
5it [00:00, 341.11it/s]
5it [00:00, 308.89it/s]
5it [00:00, 309.98it/s]
5it [00:00, 372.74it/s]
0it [00:00, ?it/s]

Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 7, 11.3] 

Посчитаем кол-во уроков в будни для класса : [8, 7, 7, 7, 7, 12.0] 

Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для кл

5it [00:00, 347.03it/s]
5it [00:00, 363.72it/s]
  0%|          | 0/400 [00:00<?, ?it/s]


Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 7, 11.3] 

Посчитаем кол-во уроков в будни для класса : [8, 7, 7, 7, 7, 12.0] 



IndexError: list index out of range

# Отображение расписания 

In [24]:
def get_timetabling(individum :list) -> pd.DataFrame:
    """
    Выводим итоговое расписание в виде датафрейма
    """
    
    step = len(CLASSROOMS)
    start_step = 0
    cols = ['day','lesson','a5','b5','a6','b6','a7','b7','a8','a9']
    df = pd.DataFrame(columns = cols)
    day = 1
    lesson = 1

    while start_step < len(individum):
        lst = []
        lst.append(day)
        lst.append(lesson)
        for key, item in enumerate(individum[start_step: start_step + step]):
               lst.append(COURSES[key][item].ItemName[0])
        
        df = pd.concat([df, pd.DataFrame([lst], columns = cols)])
        
        if lesson % 8 == 0:
            lesson = 1
            day += 1
        else:            
            lesson += 1
        start_step += step
        
    return df 

In [25]:
c = [cost(ind) for ind in d]
m = 10000
for ind, cost_ in zip(d, c):
    if cost_ < m:
        best_ind = ind
        
get_timetabling(best_ind)

,day,lesson,a5,b5,a6,b6,a7,b7,a8,a9
0,1,1,Русский,Н,Немецкий язык,Труд,Французский язык,Русский,Н,Н
0,1,2,Н,Н,Н,Французский язык,Н,Труд,География,Н
0,1,3,Н,Литература,Физкультура,Математика,Н,Алгебра,Физкультура,Н
0,1,4,Русский,Н,Немецкий язык,Литература,Физкультура,Алгебра,География,Н
0,1,5,История,Н,История,Математика,Музыка,Н,Общестовазнание,Н
0,1,6,Н,ИЗО,Н,Н,Н,Н,Музыка,Н
0,1,7,Литература,Н,Н,Русский,История,Информатика,Информатика,Н
0,1,8,Н,Русский,Н,Н,Французский язык,Литература,Алгебра,Н
0,2,1,Н,Математика,География,Русский,Алгебра,Литература,Физика,Н
0,2,2,Н,Н,Французский язык,Музыка,История,Музыка,История,Н
